This is the Colab (for GPU utilization) implementation of GCN, GAT, and GIN
============================
Usage Instructions:
  * Ensure Runtime is set to a GPU
  * As colab does not accept command line arguments, the following cell will allow user to set the most common preferences. For less commonly changed preferences, the user must modify them directly inside of the cell labled 'arguments.py File'
  * __GAN__ is more complex than GCN and some hyperparameters must be changed based on the dataset. However, the following are in common for all datasets:
    -  __two layers__ are used
    - Layer 1 has 8 attention heads with 8 features each, and its nonlinearity is the __ELU__ function
    - Dropout is applied to both layers with coefficient p = 0.6
  * GAN hyperparameters & architectural details that vary based on the dataset are as follows:
    - __Cora & CiteSeer__:  Layer 2 has 1 attention head with C (# of classes) features. L2 regularization is applied with a weighting coefficient of 0.0005
    - __PubMed__: Layer 2 has 8 attention heads with  C (# of classes) features. L2 regularization is applied with a weighting coefficient of 0.001


In [ ]:
# preferences cell
user_specified_dataset = "Cora" # choose between Cora, PubMed, and CiteSeer
user_specified_model = "GCN" # choose between GCN, GAN


In [1]:
!pip install torch_geometric > /dev/null

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [3]:
### arguments.py File ###
### Modified for colab since colab does not seem to like argparse ###

# Defined custom class to hold arguments
class Args:
  def __init__(self):
    self.root_dir = "/content"
    self.data_dir = "/content/data"
    self.epochs = 300
    self.runs = 5
    self.droput = 0.4
    self.lr = 0.001
    self.wd = 0.001
    self.num_layers = 2
    self.num_hidden = 256
    self.num_features = 0 # placeholder
    self.num_classes = 0 # placeholder

def add_data_features(args, data):
  args.num_features = data.x.shape[1]
  args.num_classes = data.y.shape[0]
  return args

In [8]:
### data.py File ###

from torch_geometric.datasets import Planetoid

args = Args()
dataset = Planetoid(root=args.root_dir, name=user_specified_dataset)

In [4]:
### model.py File ###

def make_layers(self):
    layers = []
    # initialize layers in a loop that uses conditionals to determine the input and output dimensions of the feature vectors
    for i in range(self.num_layers):
        if i == 0:  # first layer
            # dimensions in = input data size
            # dimensions out = hidden layer size
            if
            layer = GCNConv(self.num_features, self.num_hidden)

        elif i < self.num_layers - 1: # hidden layer(s)
            # dimensions in = hidden layer size
            # dimensions out = hidden layer size
            layer = GCNConv(self.num_hidden, self.num_hidden)

        else:  # output layer
            # dimensions in = hidden layer size
            # dimensions out = output size
            layer = GCNConv(self.num_hidden, self.num_classes)

        layers.append(layer)

    return nn.ModuleList(layers)

class GCN_model(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.num_features = args.num_features
        self.num_layers = args.num_layers
        self.num_hidden = args.num_hidden
        self.num_classes = args.num_classes
        self.wd = args.wd
        self.lr = args.lr
        self.layers = make_layers(self)

    def forward(self, x, edge_idx):
        for i, layer in enumerate(self.layers):
            # apply the convolutional layer
            x = layer(x, edge_idx)

            # Since I did not apply the activation function in the Layers array, I apply it using conditionals (to decide relu or softmax) here
            if i != len(self.layers) - 1:
                x = F.relu(x)
            else:
                x = F.log_softmax(x, dim = 1)

        return x

class GAN_model(nn.Module):
  def__init__(self, args):
    super().__init()
    self.num_features = args.num_features
      self.num_layers = args.num_layers
      self.num_hidden = args.num_hidden
      self.num_classes = args.num_classes
      self.wd = args.wd
      self.lr = args.lr
      self.layers = make_layers(self)


In [9]:
### main.py File ###

def train(model, X, Y, data):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr = model.lr, weight_decay = model.wd)
    optimizer.zero_grad()
    activations = model(X, data.edge_index)

    # only calculate loss on train labels!!
    loss = F.nll_loss(activations[data.train_mask], Y[data.train_mask])
    loss.backward()
    optimizer.step()

def get_masked_acc(activations, y_true, mask):
    length = activations[mask].shape[0]
    correct = 0
    for yhat, y in zip(activations[mask], y_true[mask]):
        if torch.argmax(yhat) == y:
            correct += 1

    return correct / length

def get_accuracy(activations, y_true, data):
    train_acc = get_masked_acc(activations, y_true, data.train_mask)
    test_acc = get_masked_acc(activations, y_true, data.test_mask)
    val_acc = get_masked_acc(activations, y_true, data.val_mask)
    return train_acc, test_acc, val_acc

def main():
    # use gpu if possible (works most of the time here on colab)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")

    # get data
    data = dataset[0].to(device)
    x = data.x
    y = data.y

    # get preferences
    args = Args()
    args = add_data_features(args, data)


    for run in range(args.runs):
        # initialize model
        model = GCN_model(args).to(device)
        print("\n------------ new model ------------\n")
        for epoch in range(args.epochs):
          # log loss every 50 steps
            if epoch % 50 == 0 or epoch == args.epochs - 1:
                model.eval()
                activations = model(x, data.edge_index)
                loss = F.nll_loss(activations, y)
                train_acc, test_acc, val_acc = get_accuracy(activations, y, data)
                print(f" Epoch: {epoch} | Total Loss: {loss} | Train Accuracy: {train_acc} | Test Accuracy: {test_acc} | Val Accuracy: {val_acc}")

            # backprop & update
            train(model, x, y, data)

if __name__ == '__main__':
    main()

Device: cuda

------------ new model ------------

 Epoch: 0 | Total Loss: 9.885893821716309 | Train Accuracy: 0.18333333333333332 | Test Accuracy: 0.165 | Val Accuracy: 0.136
 Epoch: 50 | Total Loss: 8.278214454650879 | Train Accuracy: 0.6333333333333333 | Test Accuracy: 0.461 | Val Accuracy: 0.476
 Epoch: 100 | Total Loss: 3.8787031173706055 | Train Accuracy: 0.6 | Test Accuracy: 0.516 | Val Accuracy: 0.538
 Epoch: 150 | Total Loss: 1.4334903955459595 | Train Accuracy: 0.65 | Test Accuracy: 0.486 | Val Accuracy: 0.516
 Epoch: 200 | Total Loss: 1.1925625801086426 | Train Accuracy: 0.43333333333333335 | Test Accuracy: 0.312 | Val Accuracy: 0.33
 Epoch: 250 | Total Loss: 0.9940095543861389 | Train Accuracy: 0.9333333333333333 | Test Accuracy: 0.734 | Val Accuracy: 0.746
 Epoch: 299 | Total Loss: 0.7798689603805542 | Train Accuracy: 0.95 | Test Accuracy: 0.769 | Val Accuracy: 0.776

------------ new model ------------

 Epoch: 0 | Total Loss: 9.88580322265625 | Train Accuracy: 0.28333333

KeyboardInterrupt: 